# Анализ банковских данных. Исследование надежности заемщиков.

## Общая информация о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных и ответы на вопросы

In [22]:
data['children'].value_counts() 
#узнаем какое максимально количество детей в семье, чтобы грамотно объединить  людей на категории многодетная/малодетная (1-2 ребенка)/нет детей

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [23]:
def categoty_family(children):
    if children == 5 or children == 4 or children == 3:
        return 'Многодетная семья'
    elif children == 2 or children == 1:
        return 'Малодетная семья'
    else:
        return 'Нет детей'

In [24]:
data['category_family'] = data['children'].apply(categoty_family)

In [25]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,category_family
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,Малодетная семья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,Малодетная семья
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,Нет детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,Многодетная семья
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,Нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью,Малодетная семья
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем,Нет детей
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью,Малодетная семья
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем,Многодетная семья


In [26]:
data_family = data.pivot_table(index = 'category_family', columns = 'debt', values = 'total_income', aggfunc = 'count')


In [27]:
data_family

debt,0,1
category_family,,
Малодетная семья,6222,638
Многодетная семья,349,31
Нет детей,13028,1063


In [28]:
data_family = data_family.reset_index()

In [29]:
#создать новый столбец, чтобы наглядно увидеть часть по формуле: 
#надо количество людей, имеющих задолжность разделить на 
#сумму людей кто имеет или не имеет задолжность по кредиту
data_family['half_with_debt'] = data_family[1] / (data_family[0] + data_family[1])
data_family

debt,category_family,0,1,half_with_debt
0,Малодетная семья,6222,638,0.093003
1,Многодетная семья,349,31,0.081579
2,Нет детей,13028,1063,0.075438


**Вывод:**
По результатам данной свобдной таблицы видно, что самая большая часть у малодетных семей, т.е. где 1 или 2 ребёнка. На втором месте идет многодетная семья, где от 3 детей до 5, а на самом последнем месте люди, у которых нет детей. Получаются самые большие трудности у людей,которых есть дети. Но разница прям минимальная, поэтому зависимости нет.

In [30]:
data_family_status = data.pivot_table(index = 'family_status', columns = 'debt', values = 'total_income', aggfunc = 'count')

In [31]:
data_family_status

debt,0,1
family_status,,
Не женат / не замужем,2523,273
в разводе,1105,84
вдовец / вдова,888,63
гражданский брак,3749,385
женат / замужем,11334,927


In [32]:
data_family_status = data_family_status.reset_index()
data_family_status

debt,family_status,0,1
0,Не женат / не замужем,2523,273
1,в разводе,1105,84
2,вдовец / вдова,888,63
3,гражданский брак,3749,385
4,женат / замужем,11334,927


In [33]:
#создать новый столбец, чтобы наглядно увидеть часть по формуле: 
#надо количество людей, имеющих задолжность разделить на 
#сумму людей кто имеет или не имеет задолжность по кредиту
data_family_status['half_with_debt'] = data_family_status[1] / (data_family_status[0] + data_family_status[1])
data_family_status

debt,family_status,0,1,half_with_debt
0,Не женат / не замужем,2523,273,0.097639
1,в разводе,1105,84,0.070648
2,вдовец / вдова,888,63,0.066246
3,гражданский брак,3749,385,0.093130
4,женат / замужем,11334,927,0.075606


**Вывод:** 
Большая доля приходится на неженатых людей и на людей, находящихся в гражданском браке. А меньшая доля на вдовцов. Можно сделать вывод, что есть зависимость между семейным положением и возвратом кредита в срок.


In [34]:
data_income = data.pivot_table(index = 'total_income_category', columns = 'debt', values = 'total_income', aggfunc = 'count')

In [35]:
data_income #Где 0–30000 — 'E'; 30001–50000 — 'D'; 50001–200000 — 'C'; 200001–1000000 — 'B'; 1000001 и выше — 'A'

debt,0,1
total_income_category,,
A,23,2
B,4660,354
C,14568,1353
D,328,21
E,20,2


In [36]:
data_income['half_with_debt'] = data_income[1] / (data_income[0] + data_income[1])
data_income

debt,0,1,half_with_debt
total_income_category,,,
A,23,2,0.080000
B,4660,354,0.070602
C,14568,1353,0.084982
D,328,21,0.060172
E,20,2,0.090909


**Вывод** Рассмотрим самые большие категории В и С. В категории С часть больше, чем в С. Тогда можно сделать вывод,что сокрее всего есть некая зависимость между уровнем дохода и возвратом кредита в срок.

In [37]:
data_purpose = data.pivot_table(index = 'purpose_category', columns = 'debt', values = 'total_income', aggfunc = 'count')

In [38]:
data_purpose

debt,0,1
purpose_category,,
операции с автомобилем,3879,400
операции с недвижимостью,9971,780
получение образования,3619,369
проведение свадьбы,2130,183


In [39]:
data_purpose['half_with_debt'] = data_purpose[1] / (data_purpose[0] + data_purpose[1])
data_purpose

debt,0,1,half_with_debt
purpose_category,,,
операции с автомобилем,3879,400,0.093480
операции с недвижимостью,9971,780,0.072551
получение образования,3619,369,0.092528
проведение свадьбы,2130,183,0.079118


**Вывод:** Большая часть должников приходится на категории: операции  автомобилем, получение образования. Я считаю, что это не связано с зависимостью на возврат кредита, так как есть и другие акспекты, рассмотренные выше.

Причины появления пропусков в исходных данных:
- 1) В результате арифметических операций, может появиться значение NaN. Там показано, что лежало какое-то вещественное число, а теперь лежит пустое значение.
- 2) None - не числовое пропущенное значение, с ним операции неосуществимы. Возникает при пропуске значения или неправильности его заполнения.
- 3) Особые обозначения, принятые создателем. Кто создал данный файл, указал пропуски, а потом забыл как указал.
- 4) Ничего не указано в ячейке данной таблицы пропустили.

 Лучше заполнять медианным значением, так как это непосредственная середина в данных. Если заполнить средним значением, оно может очень сильно отличаться от действительности (где-то может быть слишком большое или слишком маленькое значение), тогда данные сильно исказятся. А если пропуски заполнять медианным значением, то данные не изменятся и выводы будут правильные, так как это будет прям среднее значение (середина в данных).

### Общий вывод


Проанализровав полученные данные (статистику платежеспособности клиента) можно сделать вывож,что не только количество детей в семье зависит от выплат кредита в срок, а также его цель, финансовая составляющая человека, а также семейное положение.